# FIT5202 – Big Data  
## Week 3a & 3b – Parallel Join & Parallel Outer Join

---

## 0) Notation & Variable Glossary (used throughout formulas)

- **N** — number of processors.
- **P** — page size (bytes) for disk and network transfers.
- **S** — size of table **S** in bytes.
- **|S|** — number of records (tuples) in **S**.
- **Si** — size (bytes) of fragment of **S** stored at processor *i* (often `S/N` under equal partitioning).
- **|Si|** — number of records of **S** at processor *i* (often `|S|/N` under equal partitioning).
- **Ri**, **|Ri|** — analogous to **Si**, **|Si|** but for relation **R** at processor *i*.
- **IO** — time to read or write one page between disk and main memory.
- **tr** — CPU time to read one record from a memory page into a tuple buffer.
- **tw** — CPU time to write one record to a buffer (e.g., output/result buffer).
- **th** — CPU time to hash one record during hash-join build/probe.
- **tj** — CPU time to perform the actual join comparison/probe per record.
- **mp** — per-page **message protocol** handling time (software stack/driver/memcopy/ack) over the network.
- **ml** — per-page **message latency** time component (network wire + setup) on send.
- **H** — number of records (or capacity) that the in-memory hash table can hold at once.
- **σj** — **join selectivity ratio** (fraction of R×S that actually joins, used as a per-record factor).
- **πR**, **πS** — **projectivity ratios** for R and S (output bytes / input record bytes after projection).
- **M** — main-memory capacity (bytes) allocated for holding incoming data during distribution.

> Tip: Think **size in bytes** (S, Si, Ri) for **I/O and network**, and **number of records** (|S|, |Si|, |Ri|) for **CPU work**.

---

## 1) Quick Revision (from slides)

**Q1.** If a query runs on a multi-core machine, it is parallel query processing. How about if multiple *different* queries run at the same time on a multi-core machine?  
**Answer:** **Inter-query parallelism** — different queries run concurrently on different cores.

**Q2.** With **hash** data partitioning, a **discrete range** search should use how many processors?  
**Answer:** **Selected processors only** — only those owning the relevant hash buckets.

---

## 2) Join Operations (Inner vs Outer)

- **Inner Join:** returns only matching rows.
- **Outer Join:** returns matches **plus** unmatched rows from one or both sides (left/right/full).

---

## 3) Serial Join Algorithms (baseline building blocks)

1) **Nested-Loop Join:** for each record of **R**, scan all records of **S**.  
2) **Sort-Merge Join:** sort both on join key(s), then merge.  
3) **Hash-Join:** hash-partition both on the join key, then only compare within matching buckets.

---

## 4) Parallel Join — Two-Stage Pattern

1) **Data Partitioning** (move/slice data so matches co-locate).  
2) **Local Join** (apply a serial algorithm per processor).

### 4.1 Partitioning Styles

**A. Divide & Broadcast**  
- Divide one table into N disjoint slices; **broadcast** the other to all processors.  
- Choose the **smaller** table to broadcast.  
- Pro: balanced local work. Con: high network traffic for the broadcast.

**B. Disjoint Partitioning (Range/Hash)**  
- Partition **both** tables so matching keys land on the same processor.  
- Pro: avoids global broadcast. Con: can suffer **data skew** (imbalanced work).

---

## 5) Cost Models for Divide & Broadcast (with plain-English meaning)

### Phase 1 — Data Loading (each processor reads its local fragment)

**Scan cost**  
$$
\text{Scan} \;=\; \frac{S_i}{P}\times IO
$$  
*Meaning:* Disk reads happen page-by-page; time is (bytes/pages) × time per page.  
*(“**Sᵢ in bytes** divided by **page size** times **I/O cost per page**.”)*

**Select cost**  
$$
\text{Select} \;=\; |S_i|\times (t_r + t_w)
$$  
*Meaning:* CPU time to fetch each record from the page and stage it for processing/output.  
*(“**records** × (**CPU read** + **CPU write**) per record.”)*

**Slide quiz (numbers):** If \(|S|=600\) records, 100 bytes each, \(N=3\):  
\(|S_i|=200\) records and \(S_i=20{,}000\) bytes.  
*Explanation:* \(600/3=200\) records; \(200\times100=20{,}000\) bytes.

**Slide prompt:** “If \(|S|=30{,}000\) records over 3 processors, when can we proceed — after reading 30,000 or 10,000?”  
**Answer:** After each processor finishes its **local 10,000** records (the slowest processor becomes the barrier). **[Inference]** This reflects per-node local read before the broadcast phase begins.

---

### Phase 2 — Broadcast (each processor sends its Sᵢ to all others)

**Transfer (send) cost per sender**  
$$
\text{Transfer} \;=\; \frac{S_i}{P}\times (N-1)\times (m_p + m_l)
$$  
*Meaning:* Sender must transmit its pages to each of the other \(N-1\) processors; each page pays protocol + latency.  
*(“**pages sent** × **other nodes** × (**protocol**+**latency**) per page.”)*

**Receive cost per receiver**  
$$
\text{Receive} \;=\; \Bigl(\frac{S}{P}-\frac{S_i}{P}\Bigr)\times m_p
$$  
*Meaning:* A processor receives **all other fragments** of \(S\), and pays the per-page protocol overhead.  
**Why \((S/P - S_i/P)\)?** You already have your own fragment \(S_i\); you only receive the **other** pages. **[Inference]**  
**Why \(m_p\) only?** The model attributes **latency** \(m_l\) to the **sender’s** cost to avoid double-counting; receivers pay protocol handling per page but not the sender’s wire/setup latency again. **[Inference]**

**Slide quiz:** “The cost to read data from **disk** is called… ?” → **Scan cost** (not Select; Select is CPU-side).

---

### Phase 3 — Store received fragments locally

**Store cost**  
$$
\text{Store} \;=\; \Bigl(\frac{S}{P}-\frac{S_i}{P}\Bigr)\times IO
$$  
*Meaning:* Write the newly received pages of \(S\) to local disk; you don’t rewrite your own \(S_i\).  
**Why \((S/P - S_i/P)\)?** Only the **non-local** pages must be stored. **[Inference]**

---

## 6) Local Join (Hash-Join shown; same pattern for other joins)

### Phase 1 — Load Rᵢ and S

**Scan**  
$$
\text{Scan} \;=\; \Bigl(\frac{R_i}{P}+\frac{S}{P}\Bigr)\times IO
$$

**Select**  
$$
\text{Select} \;=\; (|R_i|+|S|)\times (t_r+t_w)
$$

*Meaning (both):* Disk brings in pages for \(R_i\) and \(S\); CPU stages each record for hashing/probing.

### Phase 2 — Build/Probe + Join

**Join CPU cost**  
$$
\text{Join} \;=\; |R_i|\times (t_r+t_h)\;+\;|S|\times (t_r+t_h+t_j)
$$  
*Meaning:* Build phase (read+hash) for \(R_i\); probe phase (read+hash+join comparison) for \(S\).

**Overflow (bucket spill) I/O cost**  
$$
\text{Overflow} \;=\; \Bigl(1-\min\!\bigl(\tfrac{H}{|R_i|},1\bigr)\Bigr)\times \frac{R_i}{P}\times 2\times IO
$$  
*Meaning:* If the in-memory hash can’t hold all build tuples, extra I/O occurs: **write** overflow buckets then **read** them back (hence ×2).

### Phase 3 — Write results

**Generate results (CPU)**  
$$
\text{Gen} \;=\; |R_i|\times \sigma_j \times |S|\times t_w
$$  
*Meaning:* Amount of output work scales with join selectivity \(\sigma_j\).

**Store results (disk)**  
$$
\text{Result I/O} \;=\; \frac{\pi_R\times |R_i|\times \sigma_j \times \pi_S\times |S|}{P}\times IO
$$  
*Meaning:* Projected output bytes divided by page size, times I/O per page.

---

## 7) Memory & Load-Balancing Optimizations

**Cut redundant I/O using memory \(M\)**  
If incoming distributed data can be kept in RAM through the local join, you avoid one write+read cycle:  
$$
\text{Saved I/O per node} \approx \frac{M}{P}\times IO
$$  
*Meaning:* Keep up to \(M\) bytes resident; you skip scanning that many bytes from disk later. **[Inference]**

**Skew handling**  
- Create **more** fragments than processors and **repack** to balance work.  
- With **Divide & Broadcast**, load is even; with **Disjoint** partitioning, watch for skew.

---

## 8) Parallel **Outer** Join Algorithms

### 8.1 ROJA — Redistribution Outer Join Algorithm
**Steps:** (1) redistribute both R and S on the join key; (2) local **outer** join.  
**Pros:** simple, only two steps. **Cons:** network cost + potential skew due to redistribution.

### 8.2 DOJA — Duplication Outer Join Algorithm
**Steps:** (1) replicate small table; (2) local **inner** join; (3) hash-redistribute inner-join result on attribute X; (4) local **outer** join.  
**Cons:** expensive if the “small” table isn’t actually small enough.

### 8.3 DER — Duplication & Efficient Redistribution
**Steps:** (1) broadcast **left** table; (2) local **inner** join; (3) determine **ROW IDs** of unmatched left rows; (4) redistribute **only ROW IDs**; (5) replicate ROW IDs as needed; (6) final **inner** join to attach NULLs correctly.  
**Pros:** ships **IDs** instead of full rows (lighter than DOJA). **Cons:** still pays replication if the left is large.

### 8.4 OJSO — Outer Join Skew Optimization
- Do **not** redistribute dangling (unmatched) records from the previous outer join stage.  
- Process them **locally** when possible to avoid hot spots and redundant traffic.

---

## 9) Slide Questions (inserted at their relevant spots)

- **Identify the LEFT OUTER JOIN (diagram question).**  
  **Answer:** The result that preserves **all left rows** and shows **NULLs** for non-matching right attributes (e.g., `(2,3,NULL,NULL)` in the example) is the **left outer join**. **[Inference]**

- **Why do we need to *redistribute* R and S first (multi-join example R ⟕ S ⟕ T)?**  
  **Answer:** To **co-locate matching keys** on the same processor; initial placement is not guaranteed to be co-partitioned on the join attributes, so joining without redistribution would miss matches or force remote lookups. **[Inference]**

- **Why might a shared-memory system still avoid hash-join after broadcast?**  
  **Answer:** If each processor lacks **enough working memory** to hold the hash table (build side), the local algorithm may have to switch away from pure in-memory hash-join or incur heavy spilling. **[Inference]**

- **Broadcast Receive formula prompts:**  
  – **Why \((S/P - S_i/P)\)?** You only **receive others’ pages**, not your own local pages. **[Inference]**  
  – **Why \(m_p\) only?** To avoid **double-counting latency**; the sender’s transfer cost already accounts for per-page latency \(m_l\), while the receiver pays protocol handling \(m_p\). **[Inference]**

- **Store cost prompt:**  
  – **Why \((S/P - S_i/P)\)?** You **store** only the **received** (non-local) pages, not your own. **[Inference]**

- **“Disk cost” naming quiz:**  
  – Reading from disk is **Scan cost** (I/O). CPU extraction from pages is **Select cost** (no disk).  

---

## 10) Summary

- Parallel joins = **Partition** then **Local join**.  
- **Divide & Broadcast** vs **Disjoint** partitioning drives both **network** and **I/O** patterns.  
- Cost model split:  
  - **Bytes/pages** → I/O & network terms (use \(S, S_i, R_i\)).  
  - **Records** → CPU terms (use \(|S|, |S_i|, |R_i|\)).  
- Outer joins: **ROJA, DOJA, DER**; handle skew with **OJSO**.  
- Optimize by **reducing I/O**, **avoiding unnecessary redistribution**, and **balancing load**.

---
